# General Preparations

In [7]:
import sys
sys.path.insert(0, '/yunity/arusty/PF-GAP')

#Import libraries, MDS from SKlearn, and all the custom files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
from Pipeline.functions import *


#Data
from aeon.datasets import load_gunpoint
from aeon.datasets import load_basic_motions


#Functions
from QGAP.qgap import QGAP
from Redcomets.Redcomets import REDCOMETS
from RFGAP_Rocket.RFGAP_Rocket import RFGAP_Rocket
from RDST.rdst import RDST_GAP

In [4]:
# Load Data
XTRAIN, YTRAIN = load_gunpoint(split="TRAIN")
XTEST, YTEST = load_gunpoint(split="TEST")

# XTRAIN, YTRAIN = load_basic_motions(split="TRAIN")
# XTEST, YTEST = load_basic_motions(split="TEST")

#Generate random static variables to test algorithm
XSTATIC = np.random.rand(XTRAIN.shape[0], 45)
XSTATIC_TEST = np.random.rand(XTEST.shape[0], 45)


In [8]:
#Quant
qgap = QGAP(matrix_type="dense")
quant_prox = data_to_proximities(qgap, XTRAIN, YTRAIN, XTEST, XSTATIC,XSTATIC_TEST)

#Redcomets
redcomets = REDCOMETS(variant = 3, static = XSTATIC)
reddcomets_prox = data_to_proximities(redcomets, XTRAIN, YTRAIN, XTEST, None , None)

# Rocket
rf_rocket = RFGAP_Rocket(prediction_type = "classification", rocket = "Multi",
                         n_kernels=512) # Rocket Kwargs
rocket_prox = data_to_proximities(rf_rocket, XTRAIN, YTRAIN, XTEST, XSTATIC , XSTATIC_TEST)

# RDST
rdst = RDST_GAP(save_transformed_data = True)
rdst_prox = data_to_proximities(rdst, XTRAIN, YTRAIN, None, XSTATIC , None)


# Building MDS Veiwpoints